# Some preliminary checks

In [1]:
import torch
import tensorflow as tf
import os

os.environ["RAY_DEDUP_LOGS"] = "0"

print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.1.0
CUDA Available: False
CUDA Version: None


In [2]:
import psutil

# print number of gpus / CPUs
print("Number of CPUs: ", psutil.cpu_count())

num_cpus = 12
num_gpus = 0

Number of CPUs:  12


# Training

In [3]:
from ray.rllib.policy.policy import PolicySpec
from ray.tune.registry import get_trainable_cls

from custom_env import CustomEnvironment
from config import run_config

## The RLlib configuration
class Args:
    def __init__(self):
        self.run = "PPO"
        self.framework = "torch" # "tf2" or "torch"
args = Args()

## Generate the configuration
env = CustomEnvironment(run_config["env"])

config = (
    get_trainable_cls(args.run)
    .get_default_config()
    .environment(CustomEnvironment, env_config=run_config["env"])
    .framework(args.framework)
    .training(_enable_learner_api=True, num_sgd_iter=10, sgd_minibatch_size=256, train_batch_size=20000)
    .multi_agent(
        policies= {
            "prey": PolicySpec(
                policy_class=None,  # infer automatically from Algorithm
                observation_space=env.observation_space[0],  # if None infer automatically from env
                action_space=env.action_space[0],  # if None infer automatically from env
                config={"gamma": 0.85},  # use main config plus <- this override here
            ),
            "predator": PolicySpec(
                policy_class=None,
                observation_space=env.observation_space[0],
                action_space=env.action_space[0],
                config={"gamma": 0.85},
            ),
        },
        policy_mapping_fn = lambda id, *arg, **karg: "prey" if env.agents[id].agent_type == 0 else "predator",
        policies_to_train=["prey", "predator"]
    )
    .rl_module(_enable_rl_module_api=True)
    .rollouts(
        rollout_fragment_length="auto",
        batch_mode= 'truncate_episodes',
        num_rollout_workers=num_cpus-1,
        num_envs_per_worker=1,
        #create_env_on_local_worker=False,
    )
    # This as to be specified everytime (don't know how to automatically ajust)
    .resources(
        #num_gpus = num_gpus,
        #num_gpus_per_worker=0,
        #num_cpus_per_worker=2,
        # learner workers
        #num_learner_workers=num_gpus,
        #num_gpus_per_learner_worker=1,
        #num_cpus_per_learner_worker=0,
    )
    .checkpointing(export_native_model_files=True)
)
config.exploration_config = {}



2023-10-08 23:05:01,234	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-08 23:05:01,289	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-08 23:05:01,367	WARNING deprecation.py:50 -- DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
/Users/tanguy/miniforge3/envs/collective_env/lib/python3.11/site-packages/gymnasium/spaces/box.py:130: UserWarning: WARN: Box bound precision lowered by casting to float32
  gym.logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
2023-10-08 23:05:01,680	WARNING deprecation.py:50 -- DeprecationWarning: `build_tf_policy` has been deprecated. This will raise an error in the future!
2023-10-08 23:05:01,681	WARNING deprecation.py:50 -- DeprecationWarning: `build_policy_class` has been 

In [ ]:
import ray 
from ray import air, tune
from ray.air.integrations.wandb import WandbLoggerCallback
from ray.rllib.utils.test_utils import check_learning_achieved

ray.init(num_cpus=num_cpus, num_gpus=num_gpus)

print("num CPUS rays sees :", ray.cluster_resources().get('CPU', 0))
print("num GPUS rays sees :", ray.cluster_resources().get('GPU', 0))

opti_config = {
    'stop_iters': 500,
    'stop_timesteps': 10000000,
    'stop_reward': 0.1,
    'as_test': False
}

## Run the experiemnt    
tuner = tune.Tuner(
    args.run,
    param_space=config,
    run_config=air.RunConfig(
        stop={
            "training_iteration": opti_config["stop_iters"],
            "timesteps_total": opti_config["stop_timesteps"],
            "episode_reward_mean": opti_config["stop_reward"],
        },
        verbose=3,
        callbacks=[WandbLoggerCallback(
            project="marl-rllib", 
            group="PPO",
            api_key="90dc2cefddde123eaac0caae90161981ed969abe",
            log_config=True,
        )],
        checkpoint_config=air.CheckpointConfig(
            checkpoint_at_end=True,
            checkpoint_frequency=10
        ),
    ),
)
results = tuner.fit()

if opti_config["as_test"]:
    print("Checking if learning goals were achieved")
    check_learning_achieved(results, opti_config["stop_reward"])
ray.shutdown()


2023-10-08 23:05:03,374	INFO worker.py:1642 -- Started a local Ray instance.
2023-10-08 23:05:03,817	INFO tune.py:645 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


num CPUS rays sees : 12.0
num GPUS rays sees : 0


2023-10-08 23:05:03,831	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
(pid=44998) DeprecationWarning: `DirectStepOptimizer` has been deprecated. This will raise an error in the future!
(PPO pid=44998) 2023-10-08 23:05:06,723	WARNING algorithm_config.py:2578 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config=

Trial name,agent_timesteps_total,checkpoint_dir_name,connector_metrics,counters,custom_metrics,date,done,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_total,hostname,info,iterations_since_restore,node_ip,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,pid,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,time_since_restore,time_this_iter_s,time_total_s,timers,timestamp,timesteps_total,training_iteration,trial_id
PPO_CustomEnvironment_59e38_00000,606329,,"{'ObsPreprocessorConnector_ms': 0.006701368274110736, 'StateBufferConnector_ms': 0.003612041473388672, 'ViewRequirementAgentConnector_ms': 0.4773656527201335}","{'num_env_steps_sampled': 40000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 606329, 'num_agent_steps_trained': 0}",{},2023-10-08_23-07-17,False,1000,{},-13.1752,-15.7471,-18.0806,22,33,MacBook-Pro-de-Tanguy.local,"{'learner': {'__all__': {'num_agent_steps_trained': 512.0, 'num_env_steps_trained': 273597.0, 'total_loss': 0.27068477739164504}, 'prey': {'total_loss': 0.27068477739164504, 'policy_loss': -0.0013974461996663658, 'vf_loss': 0.04968034114312784, 'vf_loss_unclipped': 0.16782034388466477, 'vf_explained_var': -0.8716005432993887, 'entropy': 2.8677008319534583, 'mean_kl_loss': 0.0067088654718796175, 'default_optimizer_lr': 5.000000000000002e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.10000000149011612}, 'predator': {'total_loss': 0.22173099595255882, 'policy_loss': -0.004570574517268409, 'vf_loss': 0.22484674687812167, 'vf_loss_unclipped': 0.7706875287265978, 'vf_explained_var': -0.546073189635432, 'entropy': 2.6696010577063003, 'mean_kl_loss': 0.007274116323971793, 'default_optimizer_lr': 5.000000000000002e-05, 'curr_lr': 5e-05, 'curr_entropy_coeff': 0.0, 'curr_kl_coeff': 0.20000000298023224}}, 'num_env_steps_sampled': 40000, 'num_env_steps_trained': 0, 'num_agent_steps_sampled': 606329, 'num_agent_steps_trained': 0}",2,127.0.0.1,606329,0,40000,20000,321.196,0,0,0,0,11,0,0,0,"{'cpu_util_percent': 25.52934782608696, 'ram_util_percent': 32.62173913043478}",44998,"{'prey': -1.0251783437068376, 'predator': 68.89013974748939}","{'prey': -4.093891080340487, 'predator': 22.83063343309827}","{'prey': -11.080116384082547, 'predator': -1.1080296546613446}","{'mean_raw_obs_processing_ms': 2.1939569612040453, 'mean_inference_ms': 1.2477843403823796, 'mean_action_processing_ms': 0.5297219811578259, 'mean_env_wait_ms': 0.5158033607569157, 'mean_env_render_ms': 0.0}",122.455,62.2698,122.455,"{'training_iteration_time_ms': 61225.002, 'sample_time_ms': 8838.516, 'synch_weights_time_ms': 7.191}",1696799237,40000,2,59e38_00000


2023-10-08 23:06:15,375	WARNING util.py:315 -- The `callbacks.on_trial_result` operation took 2.753 s, which may be a performance bottleneck.
2023-10-08 23:06:15,376	WARNING util.py:315 -- The `process_trial_result` operation took 2.754 s, which may be a performance bottleneck.
2023-10-08 23:06:15,376	WARNING util.py:315 -- Processing trial results took 2.754 s, which may be a performance bottleneck. Please consider reporting results less frequently to Ray Tune.
2023-10-08 23:06:15,376	WARNING util.py:315 -- The `process_trial_result` operation took 2.754 s, which may be a performance bottleneck.


# Render episode 

### Retrieve checkpoint

In [ ]:
best_checkpoint = results.get_best_result().checkpoint
best_checkpoint

In [ ]:
from ray.rllib.algorithms.algorithm import Algorithm

algo = Algorithm.from_checkpoint(best_checkpoint)

# After loading the algorithm
local_worker = algo.workers.local_worker()
available_policy_ids = list(local_worker.policy_map.keys())
print("Available Policy IDs:", available_policy_ids)

### Run and plot

In [ ]:
import numpy as np

def process_observations(observation, agent_ids, truncation=None):
    loc_x = [observation[key][4] if key in observation else 0 for key in agent_ids]
    loc_y = [observation[key][5] if key in observation else 0 for key in agent_ids]
    if truncation:
        still_in_the_game = [1 if not truncation[key] else 0 for key in agent_ids]
    else:
        still_in_the_game = [1 for _ in agent_ids]
    observations["loc_x"].append(np.array(loc_x))
    observations["loc_y"].append(np.array(loc_y))
    observations["still_in_the_game"].append(np.array(still_in_the_game))
    
    return observations

# Use the first available policy ID
policy_id = available_policy_ids[0]

step_count = 0
observations = {"loc_x": [], "loc_y": [], "still_in_the_game": []}

observation, _ = env.reset()
agent_ids = env._agent_ids
loc_x, loc_y, still_in_the_game = process_observations(observation, agent_ids)


while step_count < 500:
    actions = {
        key: algo.compute_single_action(
            value, policy_id="prey" if env.agents[key].agent_type == 0 else "predator"
        ) for key, value in observation.items()
    }
    
    observation, _, termination, truncation, _ = env.step(actions)
    
    observations = process_observations(observation, agent_ids, truncation)
    
    step_count += 1

stage_size = env.stage_size
observations["loc_x"] = np.array(observations["loc_x"]) * stage_size
observations["loc_y"] = np.array(observations["loc_y"]) * stage_size
observations["still_in_the_game"] = np.array(observations["still_in_the_game"])

env.close()

In [ ]:
import importlib

import animation

importlib.reload(animation)
from animation import generate_animation

ani = generate_animation(observations, env)

In [ ]:
from IPython.display import HTML

HTML(ani.to_html5_video())

# Retrain

In [ ]:
from ray.rllib.policy.policy import Policy
from ray.rllib.algorithms.callbacks import DefaultCallbacks

def restore_policy_and_weights(policy_type):
    checkpoint_path = os.path.join(best_checkpoint.to_directory(), f"policies/{policy_type}")
    restored_policy = Policy.from_checkpoint(checkpoint_path)
    return restored_policy.get_weights()

restored_policy_predator_weights = restore_policy_and_weights("predator")
restored_policy_prey_weights = restore_policy_and_weights("prey")

print("Starting new tune.Tuner().fit()")

ray.init()

# Start our actual experiment.
stop = {
    "episode_reward_mean": args.stop_reward,
    "timesteps_total": args.stop_timesteps,
    "training_iteration": args.stop_iters,
}

class RestoreWeightsCallback(DefaultCallbacks):
    def on_algorithm_init(self, *, algorithm: "Algorithm", **kwargs) -> None:
        algorithm.set_weights({"predator": restored_policy_predator_weights})
        algorithm.set_weights({"prey": restored_policy_prey_weights})

config.callbacks(RestoreWeightsCallback)

results = tune.run(
    "PPO",
    stop=stop,
    config=config.to_dict(),
    verbose=1,
)

if args.as_test:
    check_learning_achieved(results, args.stop_reward)

ray.shutdown()